In [2]:
from pylab import *
import math
import pretty_midi
import librosa             # The librosa library
import librosa.display     # librosa's display module (for plotting features)
import IPython.display     # IPython's display module (for in-line audio)
import matplotlib.pyplot as plt # matplotlib plotting functions
import matplotlib.style as ms   # plotting style
import numpy as np              # numpy numerical functions
# import aubio
# from aubio import source, pitch
ms.use('seaborn-muted')         # fancy plot designs
from __future__ import print_function # use the print() function from Python3

In [2]:
# Create a PrettyMIDI object
cello_music = pretty_midi.PrettyMIDI()
# Create an Instrument instance for a cello instrument
cello_program = pretty_midi.instrument_name_to_program('Cello')
cello = pretty_midi.Instrument(program=cello_program)
# Iterate over note names, which will be converted to note number later
for note_name in ['C5']:
    # Retrieve the MIDI note number for this note name
    note_number = pretty_midi.note_name_to_number(note_name)
    # Create a Note instance for this note, starting at 0s and ending at 3s
    note = pretty_midi.Note(velocity=100, pitch=note_number, start=0, end=3)
    # Add it to our cello instrument
    cello.notes.append(note)
# Add the cello instrument to the PrettyMIDI object
cello_music.instruments.append(cello)
# Write out the MIDI data
cello_music.write('input.mid')

In [3]:
downsample = 1
samplerate = 44100 // downsample
win_s = 4096 // downsample # fft size
hop_s = 512  // downsample # hop size
tolerance = 0.8
s = source("testo.mp3", samplerate, hop_s)
pitch_o = pitch("yin", win_s, hop_s, samplerate)
pitch_o.set_unit("midi")
pitch_o.set_tolerance(tolerance)

In [4]:
pitches = []
confidences = []

# total number of frames read
total_frames = 0
while True:
    samples, read = s()
    pitch = pitch_o(samples)[0]
    #pitch = int(round(pitch))
    confidence = pitch_o.get_confidence()
    #if confidence < 0.8: pitch = 0.
#     print("%f %f %f" % (total_frames / float(samplerate), pitch, confidence))
    pitches += [pitch]
    confidences += [confidence]
    total_frames += read
    if read < hop_s: break
pitches = np.rint(np.array(pitches)).astype(np.int)
confidences = np.array(confidences)
pitches = pitches[confidences > 0.9]
print(pitches)
pitch = np.argsort(np.bincount(pitches))[-3:]

[65 65 65 65 66 66 66 66 65 65 65 65 65 66 67 67 67 67 67 67 67 67 67 67 67
 67 67 67 67 67 67 69 69 69 69 69 69 69 69 69 69 69 69 69 69 65 65 65 65 65
 65 65 65 65 65 65 65 65 65 65 65 65 65 66 73 67 67 68 67 67 67 67 67 67 67
 67 67 67 67 67 67 72 72 72 72 72 72 72 72 72 72 72 72 72 72 73 72 67 67 67
 67 67 67 67 67 67 67 67 67 67 68 67 66 63 63 63 64 64 64 64 64 64 64 64 64
 64 64 64 64 64 65 65 65 65 65 65 65 65 65 65 65 65 65 65 65 65 65 61 61 61
 60 60 60 60 60 60 60 60 61 61 61 61 61 61 61 61 62 62 62 62 62 62 62 62 62
 62 62 62 62 62 62 62 62 63 63 63 63 61 61 61 62 62 63 63 65 65 65 65 65 65
 65 65 65 66 66 67 67 67 66 67 66 67 67 67 67 67 67 67 67 69 69 70 70 68 68
 68 70 70 70 70 70 70 70 70 70 70 70 70 70 71 71 71 71 71 70 70 70 70 71 71
 71 70 70 70 70 70 70 70 70 69 69 69 69 69 69 69 69 69 69 69 69 69 69 69 69
 69 69 69 69 69 68 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 67 65 65 65
 65  0]


In [5]:
print(pitch)
print(pretty_midi.note_name_to_number("C5"))

[69 65 67]
72


In [6]:
sr = 44100
base = 440 * 2 **(5/12)
F1 = base*2**(-1/12)
F2 = F1 * 2**(-3/12)
F3 = F2 * 2**(7/12)
F4 = F3 * 2**(-5/12)
c1 = [sin(2*pi*F1*n) * exp(-0.5*n) for n in arange(0.,2.,1./44100)] 
c2 = [sin(2*pi*F2*n) * exp(-0.5*n) for n in arange(0.,2.,1./44100)]
c3 = [sin(2*pi*F3*n) * exp(-0.5*n) for n in arange(0.,2.,1./44100)]
c4 = [sin(2*pi*F4*n) * exp(-0.5*n) for n in arange(0.,2.,1./44100)]
IPython.display.Audio(data= c1+c2+c3+c4+c1+c2+c3+c4, rate=sr) # press the "play" button to hear audio